In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
from rle_converter import rle_to_mask
from pathlib import Path

In [2]:
PROJECT_DIR = "/mnt/c/Users/jgvin/iCloudDrive/SmartMonitoring/"
LS_DATA_PATH = os.path.join(PROJECT_DIR, "Lesions")
JSON_LABEL_PATH = os.path.join(LS_DATA_PATH, "projeto_label.json")

In [17]:
with open(JSON_LABEL_PATH, "r", encoding="utf-8") as file:
  labels_data = json.load(file)


In [18]:
labels_data[0].keys()

dict_keys(['id', 'annotations', 'file_upload', 'drafts', 'predictions', 'data', 'meta', 'created_at', 'updated_at', 'inner_id', 'total_annotations', 'cancelled_annotations', 'total_predictions', 'comment_count', 'unresolved_comment_count', 'last_comment_updated_at', 'project', 'updated_by', 'comment_authors'])

In [19]:
width = labels_data[0]["annotations"][0]["result"][0]["original_width"]
height = labels_data[0]["annotations"][0]["result"][0]["original_height"]
value = labels_data[0]["annotations"][0]["result"][0]["value"]["rle"]

In [20]:
labels_data[55]["data"]

{'image': '/data/upload/2/f8e96352-JAFE_1_2.jpg'}

In [24]:
labels_data[0]["annotations"][0]["result"][0]['value']["rle"]

[0,
 75,
 0,
 0,
 57,
 27,
 255,
 255,
 255,
 0,
 255,
 255,
 224,
 31,
 255,
 252,
 3,
 255,
 255,
 128,
 127,
 255,
 240,
 15,
 255,
 254,
 1,
 255,
 255,
 192,
 63,
 255,
 248,
 7,
 255,
 255,
 0,
 255,
 255,
 224,
 31,
 255,
 252,
 3,
 255,
 255,
 128,
 127,
 255,
 240,
 15,
 255,
 254,
 1,
 255,
 255,
 192,
 63,
 255,
 248,
 7,
 255,
 255,
 0,
 255,
 255,
 224,
 31,
 255,
 252,
 3,
 255,
 255,
 128,
 127,
 255,
 240,
 15,
 255,
 254,
 1,
 255,
 255,
 192,
 63,
 255,
 248,
 7,
 164,
 83,
 0,
 195,
 104,
 19,
 144,
 112,
 236,
 176,
 8,
 216,
 35,
 159,
 197,
 255,
 241,
 223,
 198,
 129,
 195,
 172,
 192,
 35,
 96,
 142,
 2,
 59,
 252,
 127,
 255,
 29,
 252,
 100,
 28,
 58,
 124,
 2,
 50,
 8,
 231,
 242,
 191,
 252,
 108,
 28,
 58,
 92,
 2,
 56,
 12,
 199,
 255,
 28,
 7,
 14,
 143,
 0,
 142,
 255,
 53,
 255,
 198,
 65,
 195,
 161,
 192,
 35,
 159,
 205,
 255,
 241,
 239,
 198,
 65,
 195,
 159,
 192,
 35,
 191,
 206,
 255,
 241,
 192,
 112,
 231,
 176,
 8,
 239,
 243,
 223,
 252,
 1

In [51]:
labels_data[0]["annotations"][0]["result"][0]["original_width"]

960

In [53]:
mask = rle_to_mask(labels_data[0]["annotations"][0]["result"][0]['value']["rle"], labels_data[0]["annotations"][0]["result"][0]["original_height"], labels_data[0]["annotations"][0]["result"][0]["original_width"])

In [86]:
# This dictionary will hold the final results.
# Keys: full image paths.
# Values: a dictionary with keys 'image' (the loaded image) and 'annotations'
#         where 'annotations' maps brush names to binary mask numpy arrays.
results_dict = {}

# Iterate through each labeled data item.
for item in labels_data:
    # --- Retrieve the image ---
    # Assume item["data"] is a dictionary whose first key's value is the relative image path.
    image_file_key = list(item["data"].keys())[0]
    image_path = item["data"][image_file_key]

    # Create the relative image path based on your folder structure.
    path_split = image_path.split("/")
    # 'labels_folder' should be defined (e.g., labels_folder = "labels")
    rel_image_path = f"{LS_DATA_PATH}/{path_split[-2]}/{path_split[-1]}"
    img_name_short = path_split[-1]
    # Build the full path using the current working directory.
    full_image_path = os.path.join(os.getcwd(), rel_image_path)

    # Attempt to load the image.
    try:
        image = Image.open(full_image_path).convert("RGB")
        image_np = np.array(image)
    except Exception as e:
        print(f"Error loading image '{full_image_path}': {e}")
        continue  # Skip processing for this item if the image cannot be loaded.

    # If not already added, create an entry for this image.
    if full_image_path not in results_dict:
        results_dict[img_name_short] = {'image': image_np, 'annotations': {}}

    # --- Process the annotations ---
    for annotation in item.get("annotations", []):
        for result in annotation.get("result", []):
            # Check if the expected keys exist.
            if "value" in result and "rle" in result["value"]:
                rle_str = result["value"]["rle"]
                # Determine the mask dimensions from provided size or from the image dimensions.
                if "size" in result["value"]:
                    mask_shape = tuple(result["value"]["size"])  # expects [height, width]
                else:
                    mask_shape = image_np.shape[:2]

                try:
                    mask = rle_to_mask(rle_str, mask_shape[0], mask_shape[1])
                except Exception as e:
                    print(f"Error decoding RLE for image '{full_image_path}': {e}")
                    continue

                # Retrieve the brush name using the brushlabels field.
                # We take the first brush name from the list.
                if "brushlabels" in result["value"] and result["value"]["brushlabels"]:
                    brush_name = result["value"]["brushlabels"][0]
                else:
                    brush_name = "default"

                # If a mask for the same brush already exists, combine them (logical OR).
                if brush_name in results_dict[img_name_short]['annotations']:
                    existing_mask = results_dict[img_name_short]['annotations'][brush_name]
                    combined_mask = np.maximum(existing_mask, mask)
                    results_dict[img_name_short]['annotations'][brush_name] = combined_mask
                else:
                    results_dict[img_name_short]['annotations'][brush_name] = mask
            else:
                print("No RLE found in annotation result:", result)

No RLE found in annotation result: {'value': {'number': 340.972}, 'id': 'OmABR4moLg', 'from_name': 'area_manual', 'to_name': 'image', 'type': 'number', 'origin': 'manual'}
No RLE found in annotation result: {'value': {'number': 340.972}, 'id': '1JCRw_kRnU', 'from_name': 'area_manual', 'to_name': 'image', 'type': 'number', 'origin': 'manual'}
No RLE found in annotation result: {'value': {'number': 340.972}, 'id': 'vu0Xci36FG', 'from_name': 'area_manual', 'to_name': 'image', 'type': 'number', 'origin': 'manual'}
No RLE found in annotation result: {'value': {'number': 340.972}, 'id': 'vi8LHTxFkc', 'from_name': 'area_manual', 'to_name': 'image', 'type': 'number', 'origin': 'manual'}
No RLE found in annotation result: {'value': {'number': 340.972}, 'id': 'mMuwawMcsi', 'from_name': 'area_manual', 'to_name': 'image', 'type': 'number', 'origin': 'manual'}
No RLE found in annotation result: {'value': {'number': 340.972}, 'id': 'DZjRaSjjmX', 'from_name': 'area_manual', 'to_name': 'image', 'type'

In [87]:
results_dict.keys()

dict_keys(['1eab80d3-2ee8c026-dab2-412a-8fd7-62a47b193ba0.JPG', 'e77ff5d9-6bf43a67-a7a6-40b4-a4d8-8678d92dfdc4.JPG', '7dbc7985-7a20a911-79e4-45fe-9e26-a07d1d757c15.JPG', '450db769-17a3e099-01c6-4c44-b99e-8ad3832fa50a.JPG', '07f3de10-a956058c-8565-40a0-b175-5d4244d638d6.JPG', 'c2439669-b92c970c-526f-418c-a6c9-54be32098c6c.JPG', '50cf6c41-bedf85a4-149e-4a23-9216-5ee1101821fc.JPG', '2a130d6c-c1e31bdb-730f-45f3-bef3-0cf4179eb811.JPG', '774fa5d2-ce15b686-cd9c-47e5-b76d-f37638684608.JPG', 'e730ab4f-DSC_0082.JPG', 'ed625684-DSC_0088.JPG', '216cd2ae-e8d8df94-3a10-43bc-bb22-5717a273d14e.JPG', '85e57326-f69db6b4-37ed-4eb4-8d66-99feb9292d8d.JPG', 'a4add069-IMG_0129.JPEG', 'c44e2da9-IMG_0130.JPEG', '0165d062-IMG_0131.JPEG', '544c4f75-IMG_0132.JPEG', 'fb5be2b5-IMG_0133.JPEG', '2032d6f0-IMG_0134.JPEG', '245467db-IMG_0169.JPEG', 'fd4ec22b-IMG_0170.JPEG', '5c97a620-IMG_3513.JPEG', 'a799ba79-IMG_3514.JPEG', '43dee4ac-IMG_3515.JPEG', 'aec95bda-IMG_3516.JPEG', '31990b2b-IMG_4362.JPEG', '10592bc1-IMG_8717

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# For demonstration, limit to plotting a few images
num_to_plot = 15

for i, (img_path, data) in enumerate(results_dict.items()):
    if i >= num_to_plot:
        break

    image = data["image"]
    annotations = data["annotations"]

    plt.figure(figsize=(10, 10))
    plt.imshow(image)

    # Overlay each brush annotation. Using different colormap for each overlay.
    # Here we use 'jet' for all brushes; you can extend this by assigning different colormaps if needed.
    for brush, mask in annotations.items():
        # Overlay the mask (assuming mask is a 2D numpy array with binary values)
        plt.imshow(mask, alpha=0.3)

        # Find approximate center of the mask (if any pixel is marked)
        # coords = np.argwhere(mask)
        # if coords.size:
        #     center_y, center_x = coords.mean(axis=0)
        #     plt.text(center_x, center_y, brush, color='white', fontsize=12,
        #              ha='center', va='center', weight='bold')

    plt.title(f"Annotations for image: {img_path}")
    plt.axis('off')
    plt.show()

In [38]:
np.count_nonzero(mask)

np.int64(509)

In [90]:
for k,v in results_dict.items():
    print(k, v["annotations"].keys())

1eab80d3-2ee8c026-dab2-412a-8fd7-62a47b193ba0.JPG dict_keys(['lesão'])
e77ff5d9-6bf43a67-a7a6-40b4-a4d8-8678d92dfdc4.JPG dict_keys(['lesão'])
7dbc7985-7a20a911-79e4-45fe-9e26-a07d1d757c15.JPG dict_keys(['lesão', 'DE_20_30'])
450db769-17a3e099-01c6-4c44-b99e-8ad3832fa50a.JPG dict_keys(['DE_20_30', 'lesão'])
07f3de10-a956058c-8565-40a0-b175-5d4244d638d6.JPG dict_keys(['lesão', 'DE_20_30', 'DE_3_4'])
c2439669-b92c970c-526f-418c-a6c9-54be32098c6c.JPG dict_keys(['DE_20_30', 'lesão'])
50cf6c41-bedf85a4-149e-4a23-9216-5ee1101821fc.JPG dict_keys(['DE_20_30', 'lesão'])
2a130d6c-c1e31bdb-730f-45f3-bef3-0cf4179eb811.JPG dict_keys(['DE_20_30', 'lesão'])
774fa5d2-ce15b686-cd9c-47e5-b76d-f37638684608.JPG dict_keys(['DE_20_30', 'lesão'])
e730ab4f-DSC_0082.JPG dict_keys(['DE_2_2', 'lesão'])
ed625684-DSC_0088.JPG dict_keys(['DE_1.5_1.5', 'lesão'])
216cd2ae-e8d8df94-3a10-43bc-bb22-5717a273d14e.JPG dict_keys(['DE_20_30', 'lesão'])
85e57326-f69db6b4-37ed-4eb4-8d66-99feb9292d8d.JPG dict_keys(['lesão'])
a4a

In [107]:

# --- 1) Collect all classes in your dataset ---
all_classes = set()
for item in results_dict.values():
    all_classes.update(item['annotations'].keys())
# e.g. all_classes == {'lesão', 'DE_20_30', 'DE_3_4', ...}
print(f"All classes: {all_classes}")
# --- 2) Map each class to a unique integer ID ---
# Reserve 0 for background; label classes starting from 1
class_to_id = {cls: idx for idx, cls in enumerate(sorted(all_classes), start=1)}

# optional: print the mapping
print("Class → ID:", class_to_id)

# --- 3) Build your simplified dict ---
simplified = {}
for fname, item in results_dict.items():
    H, W = item['image'].shape[:2]
    seg = np.zeros((H, W), dtype=np.uint8)   # your multiclass mask
    
    for cls, mask in item['annotations'].items():
        cid = class_to_id[cls]
        # wherever mask is nonzero, set to class ID
        seg[mask > 0] = cid
    
    simplified[fname] = {
        'image':        item['image'],   # unchanged
        'annotation':   seg              # (H, W) with values in {0,…,N_classes}
    }

# Now `simplified` has the form you wanted.
# e.g. simplified['7dbc7985…'].keys() == {'image','annotation'}


All classes: {'DE_3_4', 'DE_3_1.5', 'DE_2_2', 'lesão', 'DE_1.5_1.5', 'DE_50_70', 'DE_20_30'}
Class → ID: {'DE_1.5_1.5': 1, 'DE_20_30': 2, 'DE_2_2': 3, 'DE_3_1.5': 4, 'DE_3_4': 5, 'DE_50_70': 6, 'lesão': 7}


In [113]:
simplified

{'1eab80d3-2ee8c026-dab2-412a-8fd7-62a47b193ba0.JPG': {'image': array([[[160, 114,  80],
          [160, 114,  80],
          [160, 114,  80],
          ...,
          [243, 174, 135],
          [243, 174, 135],
          [243, 174, 135]],
  
         [[160, 114,  80],
          [160, 114,  80],
          [160, 114,  80],
          ...,
          [243, 174, 135],
          [243, 174, 135],
          [243, 174, 135]],
  
         [[160, 114,  80],
          [160, 114,  80],
          [160, 114,  80],
          ...,
          [242, 173, 134],
          [242, 173, 134],
          [242, 173, 134]],
  
         ...,
  
         [[ 73,  56,  49],
          [ 73,  56,  49],
          [ 73,  56,  49],
          ...,
          [ 51,  48,  33],
          [ 49,  45,  33],
          [ 47,  43,  31]],
  
         [[ 73,  56,  49],
          [ 73,  56,  49],
          [ 73,  56,  49],
          ...,
          [ 52,  49,  34],
          [ 50,  46,  34],
          [ 49,  45,  33]],
  
         [[ 73, 

In [115]:
with open("class_id.json", 'w') as j:
    json.dump(class_to_id, j)

In [118]:
import pickle
with open("simplified_data.pkl", "wb") as p:
    pickle.dump(simplified, p)

In [3]:
import pickle
with open("/mnt/c/Users/jgvin/iCloudDrive/SmartMonitoring/Lesions/simplified_data.pkl", "rb") as p:
    simple = pickle.load(p)

In [4]:
simple

{'1eab80d3-2ee8c026-dab2-412a-8fd7-62a47b193ba0.JPG': {'image': array([[[160, 114,  80],
          [160, 114,  80],
          [160, 114,  80],
          ...,
          [243, 174, 135],
          [243, 174, 135],
          [243, 174, 135]],
  
         [[160, 114,  80],
          [160, 114,  80],
          [160, 114,  80],
          ...,
          [243, 174, 135],
          [243, 174, 135],
          [243, 174, 135]],
  
         [[160, 114,  80],
          [160, 114,  80],
          [160, 114,  80],
          ...,
          [242, 173, 134],
          [242, 173, 134],
          [242, 173, 134]],
  
         ...,
  
         [[ 73,  56,  49],
          [ 73,  56,  49],
          [ 73,  56,  49],
          ...,
          [ 51,  48,  33],
          [ 49,  45,  33],
          [ 47,  43,  31]],
  
         [[ 73,  56,  49],
          [ 73,  56,  49],
          [ 73,  56,  49],
          ...,
          [ 52,  49,  34],
          [ 50,  46,  34],
          [ 49,  45,  33]],
  
         [[ 73, 